In [115]:
from random import random, seed
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt

from icecream import ic

In [116]:
UNIVERSE_SIZE = 100_000
NUM_SETS = 10_000
DENSITY = 0.1

rng = np.random.Generator(
    np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)])
)

In [117]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = pow(SETS.sum(axis=1), 1.1)

In [118]:
valid = lambda solution: np.all(np.logical_or.reduce(SETS[solution]))
cost = lambda solution: COSTS[solution].sum()

In [119]:
# Pre-processing

tabu = []
#for i, cur_set in enumerate(SETS):
#    if cur_set.sum() == 0:
#        # empty set
#        tabu.append(i)
#plt.imshow(SETS)
ic(tabu)

ic| tabu: []


[]

In [120]:
from numpy import ceil, floor


def plot(solution):
    plt.figure(figsize=(10, 10))
    plt.imshow(SETS[solution])
    plt.show()

def acceptable(geneset):
    try:
        return not any([geneset[t] for t in tabu]) and valid(geneset)
    except TypeError:
        return False


def mutate(genesets, n: int = 1, min_mutations_percent: float = 0.1, max_mutations_percent: float = 0.1):
    """Tweaks the solution by adding/removing a random set"""
    for new_geneset in genesets:
        mutating_genes = None
        while mutating_genes is None or any(gene in tabu for gene in mutating_genes) or not valid(new_geneset):
            mutating_genes = rng.integers(0, NUM_SETS, rng.integers(ceil(UNIVERSE_SIZE*min_mutations_percent), floor(UNIVERSE_SIZE*max_mutations_percent), endpoint=True))
            new_geneset[mutating_genes] = ~new_geneset[mutating_genes]

    return genesets

MAX_RETRIES = int(sqrt(UNIVERSE_SIZE)/2) or 1
def evolve(genesets: list, n: int = 1):
    costs = [cost(g) for g in genesets]
    sort_idx = np.argsort(costs)
    new_genesets = []
    new_genesets.append(genesets[sort_idx[0]]) # Left the best one unmutated

    # Randomly cross the best geneset with the second best
    for _ in range(1, n):
        new_geneset = None
        retries = 0
        while not valid(new_geneset) and retries < MAX_RETRIES:
            retries += 1
            mask = rng.random(NUM_SETS) < (rng.uniform(0.5, 1))
            new_geneset1 = np.logical_and(genesets[sort_idx[0]], mask)
            new_geneset2 = np.logical_and(genesets[sort_idx[1]], np.logical_not(mask))
            new_geneset = np.logical_or(new_geneset1, new_geneset2)
        if retries == MAX_RETRIES:
            new_geneset = genesets[sort_idx[rng.integers(0,1,endpoint=True)]]
            # If the crossover takes too long just take one of the parents at random

        new_genesets.append(new_geneset)

    return new_genesets, costs[sort_idx[0]]

In [121]:
# Const
POPULATION_SIZE = int(sqrt(UNIVERSE_SIZE)/2) or 1
ROUNDS = 3
ic(POPULATION_SIZE)

ic| POPULATION_SIZE: 316


316

In [122]:
from IPython.display import display, clear_output
import time

fig1 = plt.figure(figsize=(10, 10))
ax1 = fig1.add_subplot(111)
#ax1.scatter([])

best_geneset_overall = None
best_cost_overall = float("inf")
best_round = 0
best_round_iterations = 0
total_iterations = 0

start = time.time()
for round in range(1, ROUNDS + 1):
    # Init
    genesets = []
    for _ in range(POPULATION_SIZE):
        geneset = rng.random(NUM_SETS) < (rng.uniform(0.4, 0.6))
        while not acceptable(geneset):
            geneset = rng.random(NUM_SETS) < (rng.uniform(0.4, 0.6))
        genesets.append(geneset)

    best_cost = min([cost(g) for g in genesets])
    last_best_cost = best_cost
    i = 0
    j = 0
    history = [best_cost]
    while True:
        i += 1
        j += 1
        mutated_genesets = mutate(genesets, n=POPULATION_SIZE, min_mutations_percent=0.01, max_mutations_percent=0.02)
        population = genesets + mutated_genesets
        genesets, best_cost = evolve(population, n=POPULATION_SIZE)

        if best_cost < last_best_cost:
            j = 0
            last_best_cost = best_cost

        history.append(last_best_cost)
        if i % 10 == 0:
            ax1.cla()
            ax1.set_title(f"Round {round}/{ROUNDS}")
            ax1.plot(history)
            display(fig1)
            clear_output(wait=True)
            plt.pause(0.1)

        if best_cost <= UNIVERSE_SIZE:
            break
        if i > UNIVERSE_SIZE*2 and j/i > 0.5:
            break
    total_iterations += i

    if last_best_cost < best_cost_overall:
        best_cost_overall = last_best_cost
        best_geneset_overall = genesets[0]
        best_round = round
        best_round_iterations = i
    if best_cost <= UNIVERSE_SIZE:
        print("Found optimal solution")
        break
elapsed = time.time() - start

In [26]:
minutes = int(elapsed // 60)
seconds = elapsed % 60
print(f"Elapsed time: {minutes}m {seconds:.2f}s")
print(f"Best cost: {best_cost_overall}")
print(f"Total number of iterations: {total_iterations}")
print(f"Best round: {best_round}{ROUNDS}")
print(f"Iterations in round {best_round}: {best_round_iterations}")

print(f"Selected sets: {list(np.nonzero(best_geneset_overall))}")
#plt.imshow(SETS[best_geneset_overall])

if not valid(best_geneset_overall):
    print("Something terrible happened!")

Elapsed time: 0m 19.81s
Best cost: 303.1047662678675
Total number of iterations: 303
Best round: 13
Iterations in round 1: 101
Selected sets: [array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]
